In [60]:
# 版本更新2.0： 全局过滤：只要7月没，就不要;
july_no_stores = ('江苏南京景枫',
 '重庆龙湖时代天街',
 '江苏苏州昆山九方城',
 '河南郑州永威木色',
 '南京华采天地',
 '四川成都银泰',
 '江苏南京印象汇',
 '廊坊新朝阳',
 '常州环球港',
 '河北保定竞秀区保定购物商场',
 '北京大兴大族广场',
 '浙江宁波万象汇',
 '杭州简易之间')

In [61]:
import pandas as pd

def df_from_sql(sql):
    import pymysql
    import pandas as pd
    connect = pymysql.Connect(
        host=polar_host,
        port=polar_port,
        user=polar_user,
        passwd=polar_pswd,
        db=polar_db
    )
    cursor = connect.cursor()
    df = pd.read_sql(sql, con=connect)
    connect.close()    
    return df

In [59]:
# 虹如要的数据；
# 1. reblue全量 7月 - 全量 6月；

sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
where sr.store_name not in {} 
'''.format(july_no_stores)

# sql_loan = '''
# select * 
# from yk_reblue_sale s
# left join yk_reblue_user u on u.user_id = s.user_id
# left join yk_reblue_saler sr on sr.saler_id = s.saler_id
# '''

blue = df_from_sql(sql_loan)

june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

print('7月总交易额{}'.format(july_blue.paid_amt.sum()))
print('6月总交易额{}'.format(june_blue.paid_amt.sum()))
print('7-6月 :{}'.format(july_blue.paid_amt.sum() - june_blue.paid_amt.sum()))


7月总交易额4060811.0
6月总交易额3453610.2800000003
7-6月 :607200.7199999997


In [62]:
# 虹如要的数据；
# 2. reblue全量 7月 客单价 总价/人数，如果非会员用dp_id代表人数；

# sql_loan = '''
# select * 
# from yk_reblue_sale s
# left join yk_reblue_user u on u.user_id = s.user_id
# '''

sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
where sr.store_name not in {} 
'''.format(july_no_stores)

blue = df_from_sql(sql_loan)
june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

july_blue = july_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]
june_blue = june_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]

june_blue = june_blue[june_blue.paid_amt>0].iloc[:,1:]
print('6月大票数：{}'.format(june_blue.dp_id.nunique()))
print('6月单均：{}'.format(june_blue.paid_amt.sum()/june_blue.dp_id.nunique()))

july_blue = july_blue[july_blue.paid_amt>0].iloc[:,1:]
print('7月大票数:{}'.format(july_blue.dp_id.nunique()))
print('7月单均:{}'.format(july_blue.paid_amt.sum()/july_blue.dp_id.nunique()))


6月大票数：2739
6月单均：1509.2360715589632
7月大票数:3165
7月单均:1486.5788372827803


In [63]:
# 虹如要的数据；
# 3. reblue全量 5.1~6.16 交易额/天数;
# sql_loan = '''
# select * 
# from yk_reblue_sale s
# '''

sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
where sr.store_name not in {} 
'''.format(july_no_stores)

blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-05-01 00:00:00', '2020-06-17 00:00:00')]
print('reblue全量 5.1~6.16 交易额/天数:{}'.format(tmp_blue.paid_amt.sum()/47))

reblue全量 5.1~6.16 交易额/天数:117455.87531914892


In [75]:
# 虹如要的数据；
# 4. reblue全量 周数据[首周22-28]；每周交易额
# 5. reblue西瓜 周数据[首周22-28]；每周交易额

# sql_loan = '''
# select * 
# from yk_reblue_sale s
# '''

sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
where sr.store_name not in {} 
'''.format(july_no_stores)

blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-06-22 00:00:00', '2020-08-01 00:00:00')]
tmp_blue = tmp_blue[['paid_amt', 'hapay_pay', 'dp_id', 'user_id', 'sale_time']]
tmp_blue['date'] = tmp_blue['sale_time'].apply(lambda x:str(x)[0:10])

def weekget(string):
    if string in ['2020-06-22','2020-06-23','2020-06-24','2020-06-25','2020-06-26','2020-06-27','2020-06-28']:
        return '上线第1周'
    elif string in ['2020-06-29','2020-06-30','2020-07-01','2020-07-02','2020-07-03','2020-07-04','2020-07-05']:
        return '上线第2周'
    elif string in ['2020-07-06','2020-07-07','2020-07-08','2020-07-09','2020-07-10','2020-07-11','2020-07-12']:
        return '上线第3周'
    elif string in ['2020-07-13','2020-07-14','2020-07-15','2020-07-16','2020-07-17','2020-07-18','2020-07-19']:
        return '上线第4周'    
    elif string in ['2020-07-20','2020-07-21','2020-07-22','2020-07-23','2020-07-24','2020-07-25','2020-07-26']:
        return '上线第5周'    
    elif string in ['2020-07-27','2020-07-28','2020-07-29','2020-07-30','2020-07-31']:
        return '上线第6周'    
    else:
        print(string)
        raise Exception('not allowed')

tmp_blue['week'] = tmp_blue.date.apply(lambda x: weekget(x))
tmp_blue.pivot_table(index='week', values=['paid_amt'], aggfunc={'paid_amt':'sum'})

,paid_amt
week,
上线第1周,842813.79
上线第2周,852319.75
上线第3周,858949.31
上线第4周,842075.35
上线第5周,1047329.33
上线第6周,613416.75


In [81]:
tmp_blue=tmp_blue[tmp_blue.hapay_pay>0]
tmp_blue = tmp_blue.pivot_table(index='dp_id', values=['hapay_pay', 'week'], aggfunc=max)
tmp_blue.pivot_table(index='week', values=['hapay_pay'], aggfunc={'hapay_pay':'sum'})

,hapay_pay
week,
上线第1周,60366.70
上线第2周,139669.85
上线第3周,108450.05
上线第4周,112167.95
上线第5周,197615.35
上线第6周,131844.70


In [8]:
# 虹如要的数据；
# 6. reblue西瓜 新老顾客
# 西瓜新老用户定义：7月1号之后来的，之前没来过都是新用户
sql_loan = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid
left join mars_user u on u.refid = l.userrefid
where f.type='交易'
'''

blue = df_from_sql(sql_loan)

blue = blue[blue.financeRefId.isin(['WFY2_Re-BLUE_default'])]
blue['flag'] = 1
user = blue.pivot_table(index='userRefId', 
                        values=['goodsOrderAmount', 'flag'],
                        aggfunc={'goodsOrderAmount':sum,
                                 'flag':sum})


old_users = blue[blue.fundTime<'2020-07-01 00:00'].userRefId
july_blue = blue[blue.fundTime>'2020-07-01 00:00']
july_blue['是老用户'] = july_blue.userRefId.isin(old_users)
# july_blue.index='userRefId', values='是老用户', aggfunc='')
h = july_blue[['userRefId', '是老用户']].drop_duplicates().set_index('userRefId')
h['是老用户'].value_counts()


/Users/cashbus/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


False    271
True      15
Name: 是老用户, dtype: int64

In [65]:
# 虹如要的数据；
# 7. reblue 门店 月份 西瓜 ---- 总量 单数 金额;
sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
where sr.store_name not in {} 
'''.format(july_no_stores)

blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-06-01 00:00:00', '2020-08-01 00:00:00')]
tmp_blue = tmp_blue[['paid_amt', 'hapay_pay', 'dp_id', 'user_id', 'sale_time', 'store_name']]
tmp_blue['month'] = tmp_blue['sale_time'].apply(lambda x:str(x)[5:7])


def get_df_smy(df):
    '''
    # 过滤掉负交易;
    # 由于xg最低单位是 单号，而yk最低单位是goods,所以不匹配要先矫正【矫正方法有待商榷】;
    1. 销售金额
    2. 总大票数目
    3. 西瓜金额
    4. 西瓜总大票数目
    '''
    # 过滤掉负交易;
    df = df[df['paid_amt']>=0]
    xg_df = df[df['hapay_pay']>0].copy()
    # 由于xg最低单位是 单号，而yk最低单位是goods,所以不匹配要先矫正;
#     xg_df.loc[:, '西瓜支付'] = xg_df.loc[:, '销售实收金额']
    amt, num = df['paid_amt'].sum(), df['dp_id'].nunique()
    xg_df2 = xg_df.pivot_table(index='dp_id', values=['hapay_pay'], aggfunc={'hapay_pay':max})
    if not xg_df2.empty:
        xg_amt, xg_num = xg_df2['hapay_pay'].sum(), xg_df2.shape[0]
    else:
        xg_amt, xg_num = 0, 0
    dic = {}
    dic['销售金额'] = amt
    dic['销售单数'] = num 
    dic['西瓜金额'] = xg_amt
    dic['西瓜单数'] = xg_num
    return dic

DIC = {}
res = []
for s in tmp_blue['store_name'].unique():
    print(s)
    tmp_df = tmp_blue[tmp_blue['store_name']==s]
    D = {}
    for m in ['06', '07']:
        dic = {}
        tmp = tmp_df[tmp_df['month']==m]
        dic['month'+ m] =  get_df_smy(tmp)
        D.update(dic)
    sub = pd.DataFrame(D).T
    DIC[s]= sub
    res.append(sub)

def sum_brackets(a):
    return sum(a, [])

class1 = sum_brackets([[i]*2 for i in list(tmp_blue['store_name'].unique())])
class2 =["month06","month07"]*len(tmp_blue['store_name'].unique())
m_index2=pd.MultiIndex.from_arrays([class1,class2],names=["门店","月份"])

df2=pd.DataFrame(pd.concat(res).values, index=m_index2, columns=['销售金额','销售单数','西瓜金额','西瓜单数'])


北京国瑞城
四川成都龙湖西宸天街
南京龙湾天街
上海宝杨宝龙
杭州滨江宝龙城店
南京中海环宇城
兰州七里河店
杭州乐堤港
江苏苏州工业园
深圳福田卓悦汇
四川成都珠江广场
杭州滨江龙湖天街
贵阳金阳南路店
河南郑州公园茂店
北京九龙山合生汇
杭州紫荆龙湖
北京朝阳金地中心店
济南恒隆
北京大兴荟聚中心店
西安大融城
西安曲江新区金地
云南南亚风情城
杭州西溪印象城
呼和浩特振华店
石家庄万达


In [66]:
df2

销售金额   销售单数       西瓜金额  西瓜单数
门店         月份                                        
北京国瑞城      month06  210004.65  132.0   13225.80   8.0
           month07  281647.10  178.0   58910.95  31.0
四川成都龙湖西宸天街 month06  132977.30  115.0       0.00   0.0
           month07  137144.55  110.0     109.30   1.0
南京龙湾天街     month06   82925.60   75.0    4524.00   5.0
           month07  118397.35   95.0    1398.80   2.0
上海宝杨宝龙     month06   84855.60   79.0    2517.20   3.0
           month07   89784.45   77.0   42950.40  23.0
杭州滨江宝龙城店   month06  208918.26  119.0    3664.70   4.0
           month07  273906.85  139.0    8597.00   6.0
南京中海环宇城    month06  150444.50   96.0       0.00   0.0
           month07   78246.30   62.0    5235.10   3.0
兰州七里河店     month06  203175.62  176.0   14614.20   8.0
           month07  276358.76  237.0  110502.45  75.0
杭州乐堤港      month06   71039.15   55.0       0.00   0.0
           month07   98347.60   80.0   12346.10   7.0
江苏苏州工业园    month06  553982.80  288.0    6269.70   3.0
           month07  506276.20  300.0   11459.80   4.0
深圳福田卓悦汇    month06  497380.70  226.0   21131.85  13.0
           month07  589615.90  301.0   33964.50  22.0
四川成都珠江广场   month06   91152.60  106.0       0.00   0.0
           month07   67803.40   65.0    1857.80   2.0
杭州滨江龙湖天街   month06  100516.80   54.0    4185.30   3.0
           month07  326033.60  200.0   15932.70   7.0
贵阳金阳南路店    month06   66058.70   39.0       0.00   0.0
           month07   50326.20   33.0   12157.70   8.0
河南郑州公园茂店   month06  116895.50   82.0       0.00   0.0
           month07  154014.50  109.0   30897.00   4.0
北京九龙山合生汇   month06  443457.52  326.0   13025.90   8.0
           month07  575065.40  425.0   96246.35  56.0
杭州紫荆龙湖     month06  140519.70   92.0   12275.00   7.0
           month07  175976.45   97.0   72189.80  29.0
北京朝阳金地中心店  month06  153434.30  100.0    8970.15   7.0
           month07  182306.00  149.0   34605.85  18.0
济南恒隆       month06  112374.50   47.0    3986.30   1.0
           month07   57464.10   44.0   11317.10   6.0
北京大兴荟聚中心店  month06  141742.00  103.0    1606.90   2.0
           month07  103222.25   85.0   53728.80  31.0
西安大融城      month06  143792.15  100.0    3526.10   1.0
           month07  161548.31  106.0   29241.65  14.0
西安曲江新区金地   month06  126024.40   83.0       0.00   0.0
           month07  143939.70   91.0   10571.10   4.0
云南南亚风情城    month06   31373.80   42.0       0.00   0.0
           month07   37351.55   32.0    4313.40   1.0
杭州西溪印象城    month06  101304.30   61.0     939.20   1.0
           month07   56552.70   32.0    4226.30   2.0
呼和浩特振华店    month06  101463.30   91.0       0.00   0.0
           month07  118033.25  108.0   15380.50   7.0
石家庄万达      month06   67983.85   70.0       0.00   0.0
           month07   45659.55   54.0    2177.90   2.0

In [67]:
df2.to_csv('/Users/cashbus/Desktop/f3.csv')

In [43]:
# 8 过滤掉7月没有西瓜交易的门店
df3 = df2.reset_index()
july_no_stores = df3[(df3['西瓜金额']==0)&(df3['月份']=='month07')]['门店'].tolist()